Original code by Zachary J. Ryan 
at CU Boulder
Spring 2020

In [8]:
#API for parsing XML docs
import xml.etree.ElementTree as ET
from itertools import chain
import sklearn
import csv



def XMLtoArray(filename, stems=False):
    '''Takes multiple FLExText texts as .xml. 
    Returns data as list: [[[[morpheme, gloss, mpos, wpos],...]word,...]sent,...]'''
    
    # Identify tiers, because the info you need may be on different tiers in FLEx.
    txt = 'txt'
    gloss = 'gls'
    cf = 'cf'
    pos = 'pos' # word-level pos
    msa = 'msa' # morpheme-level pos
    punct = 'punct'
    variantTypes = 'variantTypes'
    title_type = 'title'
    comment_type = 'comment'
    english = 'en'
    indonesian = 'id'
    stem = 'stem'
    prefix = 'prefix'
    suffix = 'suffix'
    enclitic = 'enclitic'
    phrase_type = 'phrase'
    circumfix = 'circumfix'
    infix = 'infix'
    bound_root_A = 'bound root A'
    particle = 'particle'
    bound_stem = 'bound stem'
    
    datalists = []
    #a single line for datalists is below
    #datalists -> [title, segnum, [[word_1,[[morph, gloss]], pos], [word2,[[morph, gloss], [morph, gloss]], pos], ... ,[wordn,[[morph, gloss], [morph, gloss]], pos]], translated phrase, comment]
    
    # open XML doc using xml parser
    root = ET.parse(filename).getroot()
    for lin in root.iter('interlinear-text'):
        #find title and comment if in this section
        #some documents have both and english and native language titles
        #these checks assure that the english title will always be used if both are found
        #if only one of them is found then it is used
        #if none are found return NO TITLE FOUND
        comment = "No comment"
        eng_title = "~~~"
        non_eng_title = "~~~"
        for item_lin in lin.iter('item'):
            if item_lin.get('type') == title_type and item_lin.get('lang') == english:
                eng_title = item_lin.text
            if item_lin.get('type') == title_type and item_lin.get('lang') != english:
                non_eng_title = item_lin.text
            if item_lin.get('type') == comment_type and item_lin.get('lang') == english:
                comment = item_lin.text
        if eng_title != "~~~" and non_eng_title == "~~~":
            title = eng_title
        elif eng_title == "~~~" and non_eng_title != "~~~":
            title = non_eng_title
        elif eng_title != "~~~" and non_eng_title != "~~~":
            title = eng_title
        else:
            title = "NO TITLE FOUND"
            
        #go through all paragraphs
        for paragraphs in lin.iter('paragraphs'):
            for paragraph in paragraphs.iter('paragraph'):
                #go through all phrases in paragraph
                for phrases in paragraph.iter('phrases'):
                    for phrase in phrases.iter('phrase'):
                        #create a temp array for each phrase
                        temp_line = []
                        #append title
                        temp_line.append(title)
                        #get segnum and append to temp_line
                        segnum = phrase.find('item').text
                        temp_line.append(segnum)
                        temp_words_morph_gloss = []
                        #loop through all words in the phrase
                        for words in phrase.iter('words'):
                            for word in words.iter('word'):
                                temp_word = []
                                #attach the untranslated word
                                wrd = word.find('item')
                                if wrd.get('type') == punct:
                                    temp_word.append(wrd.text)
                                    # make sure all new entries go with in both sets of brackets, 
                                    #follow the same format right after 'punct'
                                    temp_morph = [[str(wrd.text), str(wrd.text), 'punct', '', '']]
                                    temp_word.append(temp_morph)
                                    temp_word.append('punct')
                                    
                                else:
                                    temp_word.append(wrd.text.replace(' ', '~'))
                                    #find the morpheme and gloss for each word and append what is found in the xml
                                    temp_morphemes = []
                                    #stores the order of the morphemes for each word, resets each iteration of the loop
                                    #used in circumfix to decide what formatting is needed
                                    affix_order = []
                                    for morph in word.iter('morph'):
                                        #if you want to add more items that are found in each morpheme add the code here
                                        #note this needs to be done for each morph type
                                        #first add another holding place in the the temp_morph array
                                        #then add an elif statement that is of the same type already seen but instead 
                                        #check for whatever tag you want, here 'gloss' is one.
                                        #also if you do add more entries into the temp_morph you should keep it uniform
                                        #for the punctuation option above and entries to the temp morph as well.
                                        if(morph.get('type') == stem):
                                            affix_order.append(stem)
                                            temp_morph = ["~~~", "~~~", "~~~", "~~~","~~~"]
                                            for item in morph.iter('item'):
                                                if(item.get('type') == txt):
                                                    if item.text == None:
                                                        temp_morph[3] = item.text
                                                    else:
                                                        temp_morph[3] = item.text.replace('-...-','<>').replace('Ø','').replace(' ', '.')
                                                elif(item.get('type') == cf):
                                                    temp_morph[0] = item.text.replace('-...-','<>')
                                                elif(item.get('type') == gloss):
                                                    # separate multi-word glosses with "."
                                                    gloss_line = item.text
                                                    if gloss_line != None:
                                                        gloss_line = gloss_line.strip().replace(' ','.').replace('-','.')
                                                        temp_morph[1] = gloss_line
                                                elif(item.get('type') == msa):
                                                    temp_morph[2] = item.text.replace(' ', '').replace('pro-form', 'proform').replace('Nom-1','Nom1').replace('N (kx cl)', 'N.(kx.cl)')
                                                elif(item.get('type') == variantTypes):
                                                    temp_morph[4] = item.text
                                                else:
                                                    continue
                                            temp_morphemes.append(temp_morph)
                                        elif(morph.get('type') == prefix):
                                            affix_order.append(prefix)
                                            temp_morph = ["~~~", "~~~", "~~~", "~~~", "~~~"]
                                            for item in morph.iter('item'):
                                                if(item.get('type') == txt):
                                                    temp_morph[3] = item.text.replace('-...-','<>').replace('Ø','')
                                                elif(item.get('type') == cf):
                                                    if '-...-' in item.text:
                                                        #print(wrd.text)
                                                        #print(item.text)
                                                        temp_morph[0] = item.text.replace('-...-','<>') + '>-'
                                                        #print('prefix: '+temp_morph[0])
                                                    else:
                                                        temp_morph[0] = item.text.replace('-...-','<>')
                                                elif(item.get('type') == gloss):
                                                    # separate multi-word glosses with "."
                                                    gloss_line = item.text
                                                    if gloss_line != None:
                                                        gloss_line = item.text.upper()
                                                        gloss_line = gloss_line.strip().replace(' ','.').replace('-','.')
                                                        temp_morph[1] = gloss_line
                                                elif(item.get('type') == msa):
                                                    temp_morph[2] = item.text.replace(' ', '').replace('pro-form', 'proform').replace('Nom-1','Nom1').replace('N (kx cl)', 'N.(kx.cl)')
                                                elif(item.get('type') == variantTypes):
                                                    temp_morph[4] = item.text
                                                else:
                                                    continue
                                            temp_morphemes.append(temp_morph)
                                        elif(morph.get('type') == suffix):
                                            affix_order.append(suffix)
                                            temp_morph = ["~~~", "~~~", "~~~", "~~~", "~~~"]
                                            for item in morph.iter('item'):
                                                if(item.get('type') == txt):
                                                    temp_morph[3] = item.text.replace('-...-','<>').replace('Ø','')
                                                elif(item.get('type') == cf):
                                                    if '-...-' in item.text:
                                                        #print(wrd.text)
                                                        #print(item.text)
                                                        temp_morph[0] = '-<' + item.text.replace('-...-','<>')
                                                        #print('suffix: '+temp_morph[0])
                                                    else:
                                                        temp_morph[0] = item.text.replace('-...-','<>')
                                                elif(item.get('type') == gloss):
                                                    # separate multi-word glosses with "."
                                                    gloss_line = item.text
                                                    if gloss_line != None:
                                                        gloss_line = item.text.upper()
                                                        gloss_line = gloss_line.strip().replace(' ','.').replace('-','.')
                                                        temp_morph[1] = gloss_line
                                                elif(item.get('type') == msa):
                                                    temp_morph[2] = item.text.replace(' ', '').replace('pro-form', 'proform').replace('Nom-1','Nom1').replace('N (kx cl)', 'N.(kx.cl)')
                                                elif(item.get('type') == variantTypes):
                                                    temp_morph[4] = item.text
                                                else:
                                                    continue
                                            temp_morphemes.append(temp_morph)
                                        elif(morph.get('type') == enclitic):
                                            affix_order.append(enclitic)
                                            temp_morph = ["~~~", "~~~", "~~~", "~~~", "~~~"]
                                            for item in morph.iter('item'):
                                                if(item.get('type') == txt):
                                                    temp_morph[3] = '-'+item.text.replace('-...-','<>').replace('Ø','')
                                                elif(item.get('type') == cf):
                                                    temp_morph[0] = '-'+item.text.replace('-...-','<>')
                                                elif(item.get('type') == gloss):
                                                    # separate multi-word glosses with "." 
                                                    gloss_line = item.text
                                                    if gloss_line != None:
                                                        gloss_line = item.text.upper()
                                                        gloss_line = gloss_line.strip().replace(' ','.').replace('-','.')
                                                        temp_morph[1] = '-'+gloss_line
                                                elif(item.get('type') == msa):
                                                    temp_morph[2] = item.text.replace(' ', '').replace('pro-form', 'proform').replace('Nom-1','Nom1').replace('N (kx cl)', 'N.(kx.cl)')
                                                elif(item.get('type') == variantTypes):
                                                    temp_morph[4] = item.text
                                                else:
                                                    continue
                                            temp_morphemes.append(temp_morph)
                                        elif(morph.get('type') == phrase_type):
                                            affix_order.append(phrase_type)
                                            temp_morph = ["~~~", "~~~", "~~~", "~~~", "~~~"]
                                            for item in morph.iter('item'):
                                                if(item.get('type') == txt):
                                                    temp_morph[3] = item.text.replace('-...-','<>').replace('Ø','')
                                                elif(item.get('type') == cf):
                                                    temp_morph[0] = item.text.replace('-...-','<>')
                                                elif(item.get('type') == gloss):
                                                    # separate multi-word glosses with "."
                                                    gloss_line = item.text
                                                    if gloss_line != None:
                                                        gloss_line = item.text.upper()
                                                        gloss_line = gloss_line.strip().replace(' ','.').replace('-','.')
                                                        temp_morph[1] = gloss_line
                                                elif(item.get('type') == msa):
                                                    temp_morph[2] = item.text.replace(' ', '').replace('pro-form', 'proform').replace('Nom-1','Nom1').replace('N (kx cl)', 'N.(kx.cl)')
                                                elif(item.get('type') == variantTypes):
                                                    temp_morph[4] = item.text
                                                else:
                                                    continue
                                            temp_morphemes.append(temp_morph)
                                        elif(morph.get('type') == circumfix):
                                            affix_order.append(circumfix)
                                            temp_morph = ["~~~", "~~~", "~~~", "~~~", "~~~"]
                                            for item in morph.iter('item'):
                                                if(item.get('type') == txt):
                                                    #print('txt: ' + item.text)
                                                    temp_morph[3] = item.text.replace('-...-','<>').replace('Ø','')
                                                elif(item.get('type') == cf):
                                                    #if cicrumfix is the first morpheme in the word, treat similar to a prefix
                                                    #if not the first morpheme then treat it as a suffix
                                                    if affix_order[0] == circumfix:
                                                        #print('cf: ' + item.text)
                                                        #print(affix_order)
                                                        temp_morph[0] = item.text.replace('-...-','<>')+'>-'
                                                        #print(temp_morph[0])
                                                    else:
                                                        #print('cf: ' + item.text)
                                                        #print(affix_order)
                                                        temp_morph[0] = '-<'+item.text.replace('-...-','<>')
                                                        #print(temp_morph[0])
                                                elif(item.get('type') == gloss):
                                                    # separate multi-word glosses with "."
                                                    gloss_line = item.text
                                                    if gloss_line != None:
                                                        gloss_line = item.text.upper()
                                                        gloss_line = gloss_line.strip().replace(' ','.').replace('-','.')
                                                        temp_morph[1] = gloss_line
                                                elif(item.get('type') == msa):
                                                    temp_morph[2] = item.text.replace(' ', '').replace('pro-form', 'proform').replace('Nom-1','Nom1').replace('N (kx cl)', 'N.(kx.cl)')
                                                elif(item.get('type') == variantTypes):
                                                    temp_morph[4] = item.text
                                                else:
                                                    continue
                                            temp_morphemes.append(temp_morph)
                                        elif(morph.get('type') == infix):
                                            affix_order.append(infix)
                                            temp_morph = ["~~~", "~~~", "~~~", "~~~", "~~~"]
                                            for item in morph.iter('item'):
                                                if(item.get('type') == txt):
                                                    #eliminates the dashes on the outside of the text
                                                    temp_txt1 = item.text[1:-1]
                                                    temp_morph[3] = wrd.text.replace(temp_txt1, ' '+item.text+' ').replace('-...-','<>').replace('Ø','')
                                                elif(item.get('type') == cf):
                                                    #eliminates the dashes on the outside of the text
                                                    temp_txt1 = item.text[1:-1]
                                                    temp_morph[0] = wrd.text.replace(temp_txt1, ' '+item.text+' ').replace('-...-','<>').replace('Ø','')
                                                elif(item.get('type') == gloss):
                                                    # separate multi-word glosses with "."
                                                    gloss_line = item.text
                                                    if gloss_line != None:
                                                        gloss_line = item.text.upper()
                                                        gloss_line = gloss_line.strip().replace(' ','.').replace('-','.')
                                                        temp_morph[1] = gloss_line
                                                elif(item.get('type') == msa):
                                                    temp_morph[2] = item.text.replace(' ', '').replace('pro-form', 'proform').replace('Nom-1','Nom1').replace('N (kx cl)', 'N.(kx.cl)')
                                                elif(item.get('type') == variantTypes):
                                                    temp_morph[4] = item.text
                                                else:
                                                    continue
                                            temp_morphemes.append(temp_morph)
                                        elif(morph.get('type') == bound_root_A):
                                            affix_order.append(stem)
                                            temp_morph = ["~~~", "~~~", "~~~", "~~~","~~~"]
                                            for item in morph.iter('item'):
                                                if(item.get('type') == txt):
                                                    if item.text == None:
                                                        temp_morph[3] = item.text
                                                    else:
                                                        temp_morph[3] = item.text.replace('-...-','<>').replace('Ø','').replace('*', '')
                                                elif(item.get('type') == cf):
                                                    temp_morph[0] = item.text.replace('-...-','<>').replace('*', '')
                                                elif(item.get('type') == gloss):
                                                    # separate multi-word glosses with "."
                                                    gloss_line = item.text
                                                    if gloss_line != None:
                                                        gloss_line = item.text.upper()
                                                        gloss_line = gloss_line.strip().replace(' ','.').replace('-','.')
                                                        temp_morph[1] = gloss_line
                                                elif(item.get('type') == msa):
                                                    temp_morph[2] = item.text.replace(' ', '').replace('pro-form', 'proform').replace('Nom-1','Nom1').replace('N (kx cl)', 'N.(kx.cl)')
                                                elif(item.get('type') == variantTypes):
                                                    temp_morph[4] = item.text
                                                else:
                                                    continue
                                            temp_morphemes.append(temp_morph)
                                        elif(morph.get('type') == particle):
                                            affix_order.append(stem)
                                            temp_morph = ["~~~", "~~~", "~~~", "~~~","~~~"]
                                            for item in morph.iter('item'):
                                                if(item.get('type') == txt):
                                                    if item.text == None:
                                                        temp_morph[3] = item.text
                                                    else:
                                                        temp_morph[3] = item.text.replace('-...-','<>').replace('Ø','')
                                                elif(item.get('type') == cf):
                                                    temp_morph[0] = item.text.replace('-...-','<>')
                                                elif(item.get('type') == gloss):
                                                    # separate multi-word glosses with "."
                                                    gloss_line = item.text
                                                    if gloss_line != None:
                                                        gloss_line = item.text.upper()
                                                        gloss_line = gloss_line.strip().replace(' ','.').replace('-','.')
                                                        temp_morph[1] = gloss_line
                                                elif(item.get('type') == msa):
                                                    temp_morph[2] = item.text.replace(' ', '').replace('pro-form', 'proform').replace('Nom-1','Nom1').replace('N (kx cl)', 'N.(kx.cl)')
                                                elif(item.get('type') == variantTypes):
                                                    temp_morph[4] = item.text
                                                else:
                                                    continue
                                            temp_morphemes.append(temp_morph)
                                        elif(morph.get('type') == bound_stem):
                                            affix_order.append(stem)
                                            temp_morph = ["~~~", "~~~", "~~~", "~~~","~~~"]
                                            for item in morph.iter('item'):
                                                if(item.get('type') == txt):
                                                    if item.text == None:
                                                        temp_morph[3] = item.text
                                                    else:
                                                        temp_morph[3] = item.text.replace('-...-','<>').replace('Ø','')
                                                elif(item.get('type') == cf):
                                                    temp_morph[0] = item.text.replace('-...-','<>')
                                                elif(item.get('type') == gloss):
                                                    # separate multi-word glosses with "."
                                                    gloss_line = item.text
                                                    if gloss_line != None:
                                                        gloss_line = item.text.upper()
                                                        gloss_line = gloss_line.strip().replace(' ','.').replace('-','.')
                                                        temp_morph[1] = gloss_line
                                                elif(item.get('type') == msa):
                                                    temp_morph[2] = item.text.replace(' ', '').replace('pro-form', 'proform').replace('Nom-1','Nom1').replace('N (kx cl)', 'N.(kx.cl)')
                                                elif(item.get('type') == variantTypes):
                                                    temp_morph[4] = item.text
                                                else:
                                                    continue
                                            temp_morphemes.append(temp_morph)
                                        #end of morpheme checks, else statement will catch any other type and put into the general format
                                        else:
                                            print("morpheme type not checked for: " + str(morph.get('type')))
                                            temp_morph = ["~~~", "~~~", "~~~", "~~~", "~~~"]
                                            for item in morph.iter('item'):
                                                if(item.get('type') == txt):
                                                    print(item.text)
                                                    temp_morph[3] = item.text.replace('-...-','<>').replace('Ø','')
                                                elif(item.get('type') == cf):
                                                    temp_morph[0] = item.text.replace('-...-','<>')
                                                elif(item.get('type') == gloss):
                                                    # separate multi-word glosses with "."
                                                    gloss_line = item.text
                                                    if gloss_line != None:
                                                        gloss_line = item.text.upper()
                                                        gloss_line = gloss_line.strip().replace(' ','.').replace('-','.')
                                                        temp_morph[1] = gloss_line
                                                elif(item.get('type') == msa):
                                                    temp_morph[2] = item.text.replace(' ', '').replace('pro-form', 'proform').replace('Nom-1','Nom1').replace('N (kx cl)', 'N.(kx.cl)')
                                                elif(item.get('type') == variantTypes):
                                                    temp_morph[4] = item.text
                                                else:
                                                    continue
                                    temp_word.append(temp_morphemes)
                                    #look for pos for the individual word
                                    temp_pos = '~~~'
                                    for w_item in word.iter('item'):
                                        if(w_item.get('type') == pos):
                                            temp_pos = w_item.text.replace('pro-form', 'proform').replace('Nom-1','Nom1').replace('N (kx cl)', 'N.(kx.cl)')
                                        else:
                                            continue
                                    temp_word.append(temp_pos)
                                temp_words_morph_gloss.append(temp_word)
                            #finding the phrase translation, starts with a string of '~~~'
                            en_translation = '~~~'
                            id_translation = '~~~'
                            #iterate through all 'item' in branch phrase
                            temp_phrase_gloss = [p_item for p_item in phrase.iter('item')]
                            #take the last item which should be our phrase translation but the if statement checks to make sure
                            for tmg in temp_phrase_gloss:
                                if tmg.get('type') == gloss and tmg.get('lang') == english:
                                    en_translation = tmg.text
                                if tmg.get('type') == gloss and tmg.get('lang') == indonesian:
                                    id_translation = tmg.text
                            #append all the words with there morpheme and gloss array
                            temp_line.append(temp_words_morph_gloss)
                            #append the translation of the phrase to the end of the temp line
                            temp_line.append(en_translation)
                            temp_line.append(id_translation)
                            #append whatever comment may have been found
                            temp_line.append(comment)
                            #append each phrase to datalist
                            #print(temp_line)
                            datalists.append(temp_line)
    
    return datalists


def arrayToCSV(xml, langID, fileName):
    
    with open(fileName, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        for i in range(0,len(xml)):
            #starts with segnum and lang ID
            csv_line = [xml[i][1],langID]
            temp_line_phrase = ""
            temp_line_cf = ""
            temp_line_txt = ""
            temp_line_gloss = ""
            temp_line_msa = ""
            temp_line_word_pos = ""
            temp_line_variantTypes = ""
            for wmg in range(0,len(xml[i][2])):
                temp_line_phrase += (str(xml[i][2][wmg][0])+" ")
                temp_cf = ""
                temp_gloss = ""
                temp_msa = ""
                temp_txt = ""
                temp_variantTypes = ""
                temp_line_word_pos += (str(xml[i][2][wmg][2])+" ")
                for mg in range(0, len(xml[i][2][wmg][1])):
                    if mg == (len(xml[i][2][wmg][1])-1):
                        temp_cf += (str(xml[i][2][wmg][1][mg][0]))
                        temp_gloss += (str(xml[i][2][wmg][1][mg][1]))
                        temp_msa   += (str(xml[i][2][wmg][1][mg][2]))
                        temp_txt   += (str(xml[i][2][wmg][1][mg][3]))
                        temp_variantTypes += (str(xml[i][2][wmg][1][mg][4]))
                    else:
                        temp_cf += (str(xml[i][2][wmg][1][mg][0])+" ")
                        temp_gloss += (str(xml[i][2][wmg][1][mg][1])+" ")
                        temp_msa   += (str(xml[i][2][wmg][1][mg][2])+" ")
                        temp_txt   += (str(xml[i][2][wmg][1][mg][3])+" ")
                        temp_variantTypes += (str(xml[i][2][wmg][1][mg][4])+" ")
                temp_line_cf += (temp_cf+"\t")
                temp_line_gloss += (temp_gloss+"\t")
                temp_line_msa += (temp_msa+"\t")
                temp_line_txt += (temp_txt+"\t")
                temp_line_variantTypes += (temp_variantTypes+"\t")
                
                
            csv_line.append(temp_line_phrase)
            csv_line.append(temp_line_txt)
            csv_line.append(temp_line_cf)
            csv_line.append(temp_line_gloss)
            csv_line.append(xml[i][3]) #appends the english translation
            csv_line.append(xml[i][4]) #appends the indonesian translation
            csv_line.append("ML ID")
            csv_line.append(xml[i][5]) #append the comment found from file
            csv_line.append(xml[i][0]) #appends the title or Text_ID
            csv_line.append(temp_line_msa) #appends the morphemes POS
            csv_line.append(temp_line_word_pos) #appends the POS for each word
            csv_line.append(temp_line_variantTypes)
            
            writer.writerow(csv_line)

In [9]:
SOURCE_DIR = r'../'
lgs = ['btz', 'lez', 'ntu', 'mni']
#lgs = ['ntu']
for lg in lgs:
    source_file = lg + '-all_txts.flextext'
    datalists = XMLtoArray(source_file, stems=True)
    print(datalists[0])
    #outputfile = r'./CLDF/' + lg + '-Master.csv'
    outputfile = lg + '-Master.csv'
    arrayToCSV(datalists, lg, outputfile)
    

morpheme type not checked for: clitic
e
['1 - Si Layakh', '1.1', [['Alkisah', [['alkisah', 'The.story.is.told', 'n', 'alkisah', '~~~']], 'n'], [',', [[',', ',', 'punct', '', '']], 'punct'], ['ni', [['ni', 'in,.at.(space)', 'Prep', 'ni', '~~~']], 'Prep'], ['sebuah', [['sebuah', 'one', 'distrnum', 'sebuah', '~~~']], 'distrnum'], ['kute', [['kute', 'city', 'n', 'kute', '~~~']], 'n'], ['(', [['(', '(', 'punct', '', '']], 'punct'], ['kute', [['kute', 'city', 'n', 'kute', '~~~']], 'n'], ['neggekhi', [['neggekhi', 'Nagari', 'nprop', 'neggekhi', '~~~']], 'nprop'], ['desa', [['desa', 'village', 'n', 'desa', '~~~']], 'n'], [')', [[')', ')', 'punct', '', '']], 'punct'], ['megelakh', [['me-', 'TO.HAVE....', 'n>vi', 'me-', '~~~'], ['gelakh', 'title.(of.persons)', 'n', 'gelakh', '~~~']], 'vi'], ['kute', [['kute', 'city', 'n', 'kute', '~~~']], 'n'], ['kekhan', [['kekhan', 'Engkeran', 'nprop', 'kekhan', '~~~']], 'nprop'], ['si', [['si', 'relpro', 'relpro', 'si', '~~~']], 'relpro'], ['megelakh~ken', [[